In [96]:
import numpy as np
import matplotlib.pyplot as plt
import gc

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2)) 

def show_boxes_on_image(raw_image, boxes):
    plt.figure(figsize=(10,10))
    plt.imshow(raw_image)
    for box in boxes:
      show_box(box, plt.gca())
    plt.axis('on')

In [110]:
from transformers import AutoImageProcessor, AutoModelForObjectDetection
import torch
import os
from PIL import Image

def subSelectImages(paths):
    if len(paths) == 0:
        print("no filenames provided")
        return []
    
    yolo_processor = AutoImageProcessor.from_pretrained("hustvl/yolos-tiny")
    yolo_model = AutoModelForObjectDetection.from_pretrained("hustvl/yolos-tiny")

    dir_name = './images/birds'
    paths = os.listdir(dir_name)[0: 100]
    if '.DS_Store' in paths:
        paths.pop(paths.index('.DS_Store'))
    selected_filenames = []
    for path in paths:
        path = f'{dir_name}/{path}'
        if not os.path.isfile(path):
            print(f'{path} - file not found')
            return
        
        raw_image = Image.open(path).convert("RGB")
        inputs = yolo_processor(images=raw_image, return_tensors="pt")
        outputs = yolo_model(**inputs)
        target_sizes = torch.tensor([raw_image.size[::-1]])
        results = yolo_processor.post_process_object_detection(outputs, threshold=0.5, target_sizes=target_sizes)[0]
        print(f"{path} - {results['labels']}")

        # labels = results['labels']
        # if (16 in labels) or (17 in labels) or (18 in labels) or (23 in labels):
        #     selected_filenames.append(path)
        
        # if len(selected_filenames) >= 5:
        #     break

    return selected_filenames

In [111]:
subSelectImages('jaj')

# os.listdir('./images/all')

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


./images/birds/bird-1.jpeg - tensor([16])
./images/birds/bird-4.jpeg - tensor([85, 16, 16, 16])
./images/birds/bird-5.jpeg - tensor([16, 16, 16])
./images/birds/bird-2.jpeg - tensor([16, 16])
./images/birds/bird-3.jpeg - tensor([16])


[]

In [194]:
import torch
from transformers import SamModel, SamProcessor
from transformers import AutoImageProcessor, AutoModelForObjectDetection
import math
from pathlib import Path

def segmentImages(paths):
    if len(paths) == 0:
        print("no filenames provided")
        return []
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    sam_model = SamModel.from_pretrained("facebook/sam-vit-huge").to(device)
    sam_processor = SamProcessor.from_pretrained("facebook/sam-vit-huge")
    yolo_processor = AutoImageProcessor.from_pretrained("hustvl/yolos-tiny")
    yolo_model = AutoModelForObjectDetection.from_pretrained("hustvl/yolos-tiny")

    dir_name = './images/squirrels'
    paths = os.listdir(dir_name)
    if '.DS_Store' in paths:
        paths.pop(paths.index('.DS_Store'))
    masks = ''
    for index, path in enumerate(paths):
        path = f'{dir_name}/{path}'
        if not os.path.isfile(path):
            print(f'{path} - file not found')
            return
        
        raw_image = Image.open(path).convert("RGB")
        inputs = yolo_processor(images=raw_image, return_tensors="pt")
        outputs = yolo_model(**inputs)
        target_sizes = torch.tensor([raw_image.size[::-1]])
        results = yolo_processor.post_process_object_detection(outputs, threshold=0.5, target_sizes=target_sizes)[0]
        if len(results['labels']) == 0:
            print(f"{path} - No squirrels detected")
        else:
            print(f"{path} - {results['labels']}")

            input_boxes = [np.ndarray.tolist(results["boxes"].detach().numpy())]
            inputs = sam_processor(raw_image, input_boxes=input_boxes, return_tensors="pt").to(device)
            image_embeddings = sam_model.get_image_embeddings(inputs["pixel_values"])

            inputs.pop("pixel_values", None)
            inputs.update({"image_embeddings": image_embeddings})

            with torch.no_grad():
                outputs = sam_model(**inputs)
            masks = sam_processor.image_processor.post_process_masks(outputs.pred_masks.cpu(), inputs["original_sizes"].cpu(), inputs["reshaped_input_sizes"].cpu())
            masked_image = []
            for index, mask in enumerate(masks[0]):
                if not (results['labels'][index].item() >= 85):
                    masked_image = (masked_image == True) | (mask[:, :, :] == True)
            masked_image = masked_image.float()
            masks.append({'path': path, 'mask': masked_image})
            masked_image = masked_image.permute(1, 2, 0)

            # filename = path.split('/')[-1]
            # mask_path = f'./birdmask-images/{filename}'
            # mask_path = Path(mask_path)
            # if not mask_path.is_file():
            #     Path('birdmask-images').mkdir(parents=True, exist_ok=True)
            # plt.imsave(f'./birdmask-images/masked_{filename}', masked_image.numpy())
            # plt.imsave(f'./birdmask-images/{filename}', np.array(raw_image))


            filename = path.split('/')[-1]
            mask_path = f'./squirrelmask-images/{filename}'
            mask_path = Path(mask_path)
            if not mask_path.is_file():
                Path('squirrelimages').mkdir(parents=True, exist_ok=True)
            plt.imsave(f'./squirrelmask-images/masked_{filename}', masked_image.numpy())
            plt.imsave(f'./squirrelmask-images/{filename}', np.array(raw_image))
        
    # plt.show()
    return masks, results

In [ ]:
masks, results_arr = segmentImages('fsa')

# [os.listdir('./images/birds/')[2]]
# len(torch.tensor([]))

# './images/birds/bird-1.jpeg'.split('/')[-1]

# './birdmask-images/masked_bird-1.jpeg'.replace('masked_', '')

In [185]:
plt.imsave('./check.jpeg', torch.squeeze(masks[0]).permute(1, 2, 0).float().numpy())

In [ ]:
# results_arr[0]['labels'] == 16
# masks[0].float()
import torchvision.transforms as T
from PIL import Image

transform = T.ToPILImage()
mask_image = transform(torch.squeeze(masks[0].float()))
image = Image.open('./images/birds/bird-1.jpeg')
prompt = "change bird to a different kind of bird"

generateBirdFeederImagesFromText(prompt, image, mask_image)

In [94]:
import matplotlib.pyplot as plt
import torch
from diffusers import StableDiffusionInpaintPipeline


def generateBirdFeederImagesFromText(prompt, image, mask_image):
    pipeline = StableDiffusionInpaintPipeline.from_pretrained("runwayml/stable-diffusion-inpainting", torch_dtype=torch.float16)
    # pipeline = pipeline.to("cuda")

    prompt = prompt
    image = pipeline(prompt=prompt, image=image, mask_image=mask_image).images[0]
    plt.imshow(image)

In [219]:
from transformers import AutoImageProcessor, AutoModelForObjectDetection
import torch
import os
from PIL import Image
import os


def yolo(raw_image):
    yolo_processor = AutoImageProcessor.from_pretrained("hustvl/yolos-tiny")
    yolo_model = AutoModelForObjectDetection.from_pretrained("hustvl/yolos-tiny")

    inputs = yolo_processor(images=raw_image, return_tensors="pt")
    outputs = yolo_model(**inputs)
    target_sizes = torch.tensor([raw_image.size[::-1]])
    results = yolo_processor.post_process_object_detection(outputs, threshold=0.5, target_sizes=target_sizes)[0]
    return results


def subSelectBirdImages(paths):
    if len(paths) == 0:
        print("empty input")
        return []
    dir_name = './images/squirrels'
    paths = os.listdir(dir_name)
    if '.DS_Store' in paths:
        paths.pop(paths.index('.DS_Store'))
    selected_filenames = []
    for path in paths:
        path = f'{dir_name}/{path}'
        if not os.path.isfile(path):
            print(f'{path} - file not found')
            return
        
        print(path)
        raw_image = Image.open(path).convert("RGB")
        results = yolo(raw_image=raw_image)
        labels = results['labels']
        if (17 in labels) or (18 in labels) or (20 in labels) or (23 in labels) or (25 in labels) or (44 in labels) or (85 in labels) or (86 in labels):
            selected_filenames.append(path)
        
        if len(selected_filenames) >= 5:
            break
    return selected_filenames

In [ ]:
subSelectBirdImages('sda')

# raw_image = Image.open('./test.jpeg').convert("RGB")
# yolo(raw_image=raw_image)['labels']

In [231]:
from diffusers import StableDiffusionInpaintPipeline

def stable_diffusion(prompt, raw_image, mask_image):
    pipeline = StableDiffusionInpaintPipeline.from_pretrained("runwayml/stable-diffusion-inpainting", torch_dtype=torch.float32, safety_checker = None,
    requires_safety_checker = False)
    pipeline.to('cuda')
    image = pipeline(prompt=prompt, image=raw_image, mask_image=mask_image).images[0]
    return image

prompt = "remove the mask area in the image and fill the area with background"
path = '../HW-3/sv695/squirrelmask-images/masked_squirrel-1.jpeg'
raw_image = Image.open(path.replace('masked_', '')).convert("RGB").resize((512, 512))
masked_image = Image.open(path).convert("RGB").resize((512, 512))
squirrel_removed = stable_diffusion(prompt=prompt, raw_image=raw_image, mask_image=masked_image)

rm_dir_name = './squirrels-removed'
filename = path.split('/')[-1]
mask_path = f'./{rm_dir_name}/{filename}'
mask_path = Path(mask_path)
if not mask_path.is_file():
    Path(rm_dir_name).mkdir(parents=True, exist_ok=True)
new_filename = f"{filename.split('.jpeg')[0]}-squirrelsRemoved.jpeg"
plt.imsave(f'./{rm_dir_name}/{new_filename}', np.array(squirrel_removed))
plt.imsave(f'./{rm_dir_name}/raw_image_of-{new_filename}', np.array(raw_image))

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
100%|██████████| 50/50 [03:32<00:00,  4.25s/it]
